# day 10

https://adventofcode.com/10/day/10

In [ ]:
import logging
import logging.config
import os

import numpy as np
import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day10.txt')

LOGGER = logging.getLogger('day10')

## part 1

### problem statement:

#### loading data

In [ ]:
test_data = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"""

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read()

In [ ]:
T_IL_DIAGRAM = int
T_BUTTON = int
T_BUTTON_LIST = list[T_BUTTON]
T_JOLTAGE_REQ = list[int]
T_MACHINE = tuple[T_IL_DIAGRAM, T_BUTTON_LIST, T_JOLTAGE_REQ]
T_MANUAL = list[T_MACHINE]


def parse_il_diagram(il_diagram_str: str) -> T_IL_DIAGRAM:
    """treat the encoding as a binary string representation"""
    return int(il_diagram_str[1: -1].replace('.', '0').replace('#', '1'), 2)

assert parse_il_diagram(il_diagram_str="[.##.]") == 6
assert parse_il_diagram(il_diagram_str="[...#.]") == 2
assert parse_il_diagram(il_diagram_str="[.###.#]") == 29


def unparse_il_diagram(il_diagram: T_IL_DIAGRAM, num_switches: int) -> str:
    """make the string representation just to verify that we're not losing our minds"""
    return f'{il_diagram:0{num_switches}b}'.replace('0', '.').replace('1', '#')


assert unparse_il_diagram(il_diagram=0, num_switches=4) == "...."
assert unparse_il_diagram(il_diagram=6, num_switches=4) == ".##."
assert unparse_il_diagram(il_diagram=6, num_switches=5) == "..##."
assert unparse_il_diagram(il_diagram=12, num_switches=5) == ".##.."
assert unparse_il_diagram(il_diagram=18, num_switches=5) == "#..#."
assert unparse_il_diagram(il_diagram=9, num_switches=5) == ".#..#"


def parse_button_press(button_press_str: str, num_switches: int) -> T_BUTTON:
    on_indices = [int(n) for n in button_press_str[1: -1].split(',')]
    return int(''.join([str(int(i in on_indices)) for i in range(num_switches)]),
               2)

assert parse_button_press(button_press_str="(1,3)", num_switches=4) == parse_il_diagram(il_diagram_str="[.#.#]")
assert parse_button_press(button_press_str="(2)", num_switches=4) == parse_il_diagram(il_diagram_str="[..#.]")
assert parse_button_press(button_press_str="(0)", num_switches=4) == parse_il_diagram(il_diagram_str="[#...]")
assert parse_button_press(button_press_str="(0, 3)", num_switches=4) == parse_il_diagram(il_diagram_str="[#..#]")
assert parse_button_press(button_press_str="(0, 3)", num_switches=5) == parse_il_diagram(il_diagram_str="[#..#.]")
assert parse_button_press(button_press_str="(1, 4)", num_switches=5) == parse_il_diagram(il_diagram_str="[.#..#]")


def parse_button_presses(button_press_strs: list[str], num_switches: int) -> T_BUTTON_LIST:
    return [parse_button_press(button_press_str=button_press_str, num_switches=num_switches)
            for button_press_str in button_press_strs]

assert parse_button_presses(button_press_strs=["(1,3)", "(2)", "(0)", "(0, 3)"], num_switches=4) == [
    parse_il_diagram(il_diagram_str="[.#.#]"),
    parse_il_diagram(il_diagram_str="[..#.]"),
    parse_il_diagram(il_diagram_str="[#...]"),
    parse_il_diagram(il_diagram_str="[#..#]"),
]

In [ ]:
def parse_raw_data(data: str) -> T_MANUAL:
    manual = []
    for line in data.strip().split('\n'):
        il_diagram_str, *button_press_strs, joltage_str = line.split(' ')
        il_diagram = parse_il_diagram(il_diagram_str=il_diagram_str)
        button_presses = parse_button_presses(button_press_strs=button_press_strs, num_switches=len(il_diagram_str) - 2)
        joltage = [int(_) for _ in joltage_str[1: -1].split(',')]
        manual.append((il_diagram, button_presses, joltage))
    return manual

In [ ]:
parse_raw_data(data=test_data)

#### function def

In [ ]:
def fewest_button_presses(machine: T_MACHINE, max_steps: int | None = None) -> int:
    il_diagram, button_presses, joltage = machine

    seen_nodes = {0, }
    just_seen_nodes = {0, }
    steps_taken = 0
    while True:
        steps_taken += 1

        if max_steps is not None and steps_taken > max_steps:
            raise ValueError('max steps exceeded')

        new_just_seen_nodes = set()
        for src in just_seen_nodes:
            for step in button_presses:
                new_node = src ^ step

                if new_node == il_diagram:
                    return steps_taken
                elif new_node in seen_nodes:
                    continue
                else:
                    seen_nodes.add(new_node)
                    new_just_seen_nodes.add(new_node)
        just_seen_nodes = new_just_seen_nodes

In [ ]:
even_testier_data = """[.##.] (0,2) (0,1) {3,5,4,7}"""
manual = parse_raw_data(data=even_testier_data)
assert fewest_button_presses(machine=manual[0], max_steps=5) == 2

In [ ]:
manual = parse_raw_data(data=test_data)

assert fewest_button_presses(machine=manual[0], max_steps=5) == 2
assert fewest_button_presses(machine=manual[1], max_steps=5) == 3
assert fewest_button_presses(machine=manual[2], max_steps=5) == 2

In [ ]:
def q_1(data):
    manual = parse_raw_data(data=data)
    return sum(fewest_button_presses(machine=machine) for machine in manual)

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == 7
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

In [ ]:
T_BUTTON_2 = int
T_BUTTON_LIST_2 = list[T_BUTTON_2]
T_MACHINE_2 = tuple[T_BUTTON_LIST_2, T_JOLTAGE_REQ]
T_MANUAL_2 = list[T_MACHINE_2]

def parse_raw_data_2(data: str) -> T_MANUAL_2:
    manual = []
    for line in data.strip().split('\n'):
        _, *button_press_strs, joltage_str = line.split(' ')
        button_presses = [[int(_) for _ in button_press_str[1: -1].split(',')] for button_press_str in button_press_strs]
        joltage = [int(_) for _ in joltage_str[1: -1].split(',')]
        manual.append((button_presses, joltage))
    return manual

In [ ]:
parse_raw_data_2(data=test_data)

#### function def

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, LpInteger, LpStatus, PULP_CBC_CMD, value

# (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}

# Create the problem
prob = LpProblem("aoc.10.2", LpMinimize)

# Define integer variables
a = LpVariable("a", 0, None, LpInteger)
b = LpVariable("b", 0, None, LpInteger)
c = LpVariable("c", 0, None, LpInteger)
d = LpVariable("d", 0, None, LpInteger)
e = LpVariable("e", 0, None, LpInteger)
f = LpVariable("f", 0, None, LpInteger)

# Define the objective function
prob += sum([a, b, c, d, e, f])
# prob += a + b + c + d + e + f

# Define constraints
# prob += e + f == 3
prob += sum([e, f]) == 3
prob += b + f == 5
prob += c + d + e == 4
prob += a + b + d == 7

# Solve the problem
prob.solve(PULP_CBC_CMD(msg=False))

assert LpStatus[prob.status] == 'Optimal'
value(prob.objective)

In [ ]:
from pulp import LpProblem, LpVariable, LpInteger, LpStatus, PULP_CBC_CMD, value

def fewest_button_presses_2(machine: T_MACHINE_2) -> int:
    button_presses, joltage = machine

    prob = LpProblem()
    lpvars = [LpVariable(f"v{i}", 0, None, LpInteger) for (i, bp) in enumerate(button_presses)]
    prob += sum(lpvars)
    for (j, joltage_val) in enumerate(joltage):
        constraint = sum([lpv
                          for (lpv, bp) in zip(lpvars, button_presses)
                          if j in bp]) == joltage_val
        prob += constraint

    prob.solve(PULP_CBC_CMD(msg=False))

    assert LpStatus[prob.status] == 'Optimal'
    return value(prob.objective)

In [ ]:
manual = parse_raw_data_2(data=test_data)

assert fewest_button_presses_2(machine=manual[0]) == 10
assert fewest_button_presses_2(machine=manual[1]) == 12
assert fewest_button_presses_2(machine=manual[2]) == 11

In [ ]:
def q_2(data):
    manual = parse_raw_data_2(data=data)
    return sum(fewest_button_presses_2(machine=machine) for machine in manual)

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == 33
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin